## Credit Card Fraud Detection Using Neural Network

In [1]:
import pandas as pd

In [4]:
data = pd.read_csv('../data/fraudTrain.csv')
data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [5]:
pd.options.display.max_columns = None
data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [6]:
data.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

In [10]:
# 1. Dropping the unwanted columns
data.drop(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant',
           'first', 'last', 'street', 'city', 'zip', 'lat', 'long', 'dob',
           'trans_num', 'unix_time', 'merch_lat', 'merch_long'], inplace=True, axis=1
        )

In [11]:
data.head()

,category,amt,gender,state,city_pop,job,is_fraud
0,misc_net,4.97,F,NC,3495,"Psychologist, counselling",0
1,grocery_pos,107.23,F,WA,149,Special educational needs teacher,0
2,entertainment,220.11,M,ID,4154,Nature conservation officer,0
3,gas_transport,45.00,M,MT,1939,Patent attorney,0
4,misc_pos,41.96,M,VA,99,Dance movement psychotherapist,0


In [18]:
# 2. Checking for missing values
data.isnull().sum()

category    0
amt         0
gender      0
state       0
city_pop    0
job         0
is_fraud    0
dtype: int64

In [21]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 7 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   category  1296675 non-null  object 
 1   amt       1296675 non-null  float64
 2   gender    1296675 non-null  object 
 3   state     1296675 non-null  object 
 4   city_pop  1296675 non-null  int64  
 5   job       1296675 non-null  object 
 6   is_fraud  1296675 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 69.3+ MB
None


In [22]:
# 3. converting categorical values to numerical values
data['category'].unique()

array(['misc_net', 'grocery_pos', 'entertainment', 'gas_transport',
       'misc_pos', 'grocery_net', 'shopping_net', 'shopping_pos',
       'food_dining', 'personal_care', 'health_fitness', 'travel',
       'kids_pets', 'home'], dtype=object)

In [25]:
category_df = (pd.get_dummies(data['category'],)).astype(int) # this is one hot encoding using pandas with get_dummies

In [26]:
category_df.head()

,entertainment,food_dining,gas_transport,grocery_net,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [28]:
data.drop('category', inplace=True, axis=1)

In [29]:
data.head()

,amt,gender,state,city_pop,job,is_fraud
0,4.97,F,NC,3495,"Psychologist, counselling",0
1,107.23,F,WA,149,Special educational needs teacher,0
2,220.11,M,ID,4154,Nature conservation officer,0
3,45.00,M,MT,1939,Patent attorney,0
4,41.96,M,VA,99,Dance movement psychotherapist,0


In [30]:
df = pd.concat([data, category_df], axis=1)

In [31]:
df.head()

,amt,gender,state,city_pop,job,is_fraud,entertainment,food_dining,gas_transport,grocery_net,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel
0,4.97,F,NC,3495,"Psychologist, counselling",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,107.23,F,WA,149,Special educational needs teacher,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,220.11,M,ID,4154,Nature conservation officer,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,45.00,M,MT,1939,Patent attorney,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,41.96,M,VA,99,Dance movement psychotherapist,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [32]:
df['gender'].unique()

array(['F', 'M'], dtype=object)

In [33]:
df['gender'].replace({'M': 0, 'F': 1}, inplace=True)

C:\Users\USer\AppData\Local\Temp\ipykernel_25028\772758906.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['gender'].replace({'M': 0, 'F': 1}, inplace=True)


In [34]:
df['gender'].unique()

array([1, 0], dtype=int64)

In [35]:
df['state'].unique()

array(['NC', 'WA', 'ID', 'MT', 'VA', 'PA', 'KS', 'TN', 'IA', 'WV', 'FL',
       'CA', 'NM', 'NJ', 'OK', 'IN', 'MA', 'TX', 'WI', 'MI', 'WY', 'HI',
       'NE', 'OR', 'LA', 'DC', 'KY', 'NY', 'MS', 'UT', 'AL', 'AR', 'MD',
       'GA', 'ME', 'AZ', 'MN', 'OH', 'CO', 'VT', 'MO', 'SC', 'NV', 'IL',
       'NH', 'SD', 'AK', 'ND', 'CT', 'RI', 'DE'], dtype=object)

In [36]:
state_df = (pd.get_dummies(df['state'])).astype(int)

In [37]:
state_df.head()

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [38]:
df.drop('state', inplace=True, axis=1)

In [39]:
df.head()

,amt,gender,city_pop,job,is_fraud,entertainment,food_dining,gas_transport,grocery_net,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel
0,4.97,1,3495,"Psychologist, counselling",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,107.23,1,149,Special educational needs teacher,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,220.11,0,4154,Nature conservation officer,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,45.00,0,1939,Patent attorney,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,41.96,0,99,Dance movement psychotherapist,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [41]:
df = pd.concat([df, state_df], axis=1)
df.head()

,amt,gender,city_pop,job,is_fraud,entertainment,food_dining,gas_transport,grocery_net,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,4.97,1,3495,"Psychologist, counselling",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,107.23,1,149,Special educational needs teacher,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,220.11,0,4154,Nature conservation officer,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,45.00,0,1939,Patent attorney,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,41.96,0,99,Dance movement psychotherapist,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [43]:
df['job'].unique().shape

(494,)

In [44]:
job_df = (pd.get_dummies(df['job'])).astype(int)

In [45]:
job_df.head()

,Academic librarian,"Accountant, chartered","Accountant, chartered certified","Accountant, chartered public finance",Accounting technician,Acupuncturist,Administrator,"Administrator, arts","Administrator, charities/voluntary organisations","Administrator, education","Administrator, local government",Advertising account executive,Advertising account planner,Advertising copywriter,Advice worker,Aeronautical engineer,Agricultural consultant,Aid worker,Air broker,Air cabin crew,Air traffic controller,Airline pilot,Ambulance person,Amenity horticulturist,Analytical chemist,Animal nutritionist,Animal technologist,Animator,Applications developer,Arboriculturist,Archaeologist,Architect,Architectural technologist,Archivist,Armed forces logistics/support/administrative officer,Armed forces technical officer,Armed forces training and education officer,Art gallery manager,Art therapist,Artist,Arts development officer,Associate Professor,Audiological scientist,Barista,Barrister,Barrister's clerk,"Biochemist, clinical",Biomedical engineer,Biomedical scientist,Bookseller,Broadcast engineer,Broadcast journalist,Broadcast presenter,Building control surveyor,Building services engineer,Building surveyor,"Buyer, industrial","Buyer, retail",Cabin crew,Call centre manager,Camera operator,Careers adviser,Careers information officer,Cartographer,Catering manager,Ceramics designer,Charity fundraiser,Charity officer,Chartered accountant,Chartered legal executive (England and Wales),Chartered loss adjuster,Chartered public finance accountant,Chemical engineer,"Chemist, analytical",Chief Executive Officer,Chief Financial Officer,Chief Marketing Officer,Chief Operating Officer,Chief Strategy Officer,Chief Technology Officer,Chief of Staff,Child psychotherapist,Chiropodist,Civil Service administrator,Civil Service fast streamer,"Civil engineer, contracting",Claims inspector/assessor,Clinical biochemist,Clinical cytogeneticist,Clinical psychologist,Clinical research associate,Clothing/textile technologist,Colour technologist,Commercial horticulturist,Commercial/residential surveyor,Commissioning editor,Communications engineer,Community arts worker,Community development worker,Community education officer,Community pharmacist,Company secretary,Comptroller,"Conservation officer, historic buildings","Conservator, furniture","Conservator, museum/gallery",Contracting civil engineer,Contractor,Control and instrumentation engineer,Copy,"Copywriter, advertising",Corporate investment banker,Counselling psychologist,Counsellor,Curator,Cytogeneticist,Dance movement psychotherapist,Dancer,Data scientist,Database administrator,Dealer,"Designer, ceramics/pottery","Designer, exhibition/display","Designer, furniture","Designer, industrial/product","Designer, interior/spatial","Designer, jewellery","Designer, multimedia","Designer, television/film set","Designer, textile","Development worker, community","Development worker, international aid",Diagnostic radiographer,Dispensing optician,"Doctor, general practice","Doctor, hospital",Drilling engineer,Early years teacher,Ecologist,Economist,"Editor, commissioning","Editor, film/video","Editor, magazine features",Education administrator,"Education officer, community","Education officer, museum",Educational psychologist,Electrical engineer,Electronics engineer,"Embryologist, clinical",Emergency planning/management officer,Energy engineer,Energy manager,"Engineer, aeronautical","Engineer, agricultural","Engineer, automotive","Engineer, biomedical","Engineer, broadcasting (operations)","Engineer, building services","Engineer, civil (consulting)","Engineer, civil (contracting)","Engineer, communications","Engineer, control and instrumentation","Engineer, drilling","Engineer, electronics","Engineer, land","Engineer, maintenance","Engineer, manufacturing","Engineer, materials","Engineer, mining","Engineer, petroleum","Engineer, production","Engineer, site","Engineer, structural","Engineer, technical sales",Engineering geologist,Englis

In [46]:
df.drop('job', inplace=True, axis=1)

In [48]:
df = pd.concat([df, job_df], axis=1)

In [49]:
df.shape

(1296675, 614)

In [50]:
df.head()

,amt,gender,city_pop,is_fraud,entertainment,food_dining,gas_transport,grocery_net,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,Academic librarian,"Accountant, chartered","Accountant, chartered certified","Accountant, chartered public finance",Accounting technician,Acupuncturist,Administrator,"Administrator, arts","Administrator, charities/voluntary organisations","Administrator, education","Administrator, local government",Advertising account executive,Advertising account planner,Advertising copywriter,Advice worker,Aeronautical engineer,Agricultural consultant,Aid worker,Air broker,Air cabin crew,Air traffic controller,Airline pilot,Ambulance person,Amenity horticulturist,Analytical chemist,Animal nutritionist,Animal technologist,Animator,Applications developer,Arboriculturist,Archaeologist,Architect,Architectural technologist,Archivist,Armed forces logistics/support/administrative officer,Armed forces technical officer,Armed forces training and education officer,Art gallery manager,Art therapist,Artist,Arts development officer,Associate Professor,Audiological scientist,Barista,Barrister,Barrister's clerk,"Biochemist, clinical",Biomedical engineer,Biomedical scientist,Bookseller,Broadcast engineer,Broadcast journalist,Broadcast presenter,Building control surveyor,Building services engineer,Building surveyor,"Buyer, industrial","Buyer, retail",Cabin crew,Call centre manager,Camera operator,Careers adviser,Careers information officer,Cartographer,Catering manager,Ceramics designer,Charity fundraiser,Charity officer,Chartered accountant,Chartered legal executive (England and Wales),Chartered loss adjuster,Chartered public finance accountant,Chemical engineer,"Chemist, analytical",Chief Executive Officer,Chief Financial Officer,Chief Marketing Officer,Chief Operating Officer,Chief Strategy Officer,Chief Technology Officer,Chief of Staff,Child psychotherapist,Chiropodist,Civil Service administrator,Civil Service fast streamer,"Civil engineer, contracting",Claims inspector/assessor,Clinical biochemist,Clinical cytogeneticist,Clinical psychologist,Clinical research associate,Clothing/textile technologist,Colour technologist,Commercial horticulturist,Commercial/residential surveyor,Commissioning editor,Communications engineer,Community arts worker,Community development worker,Community education officer,Community pharmacist,Company secretary,Comptroller,"Conservation officer, historic buildings","Conservator, furniture","Conservator, museum/gallery",Contracting civil engineer,Contractor,Control and instrumentation engineer,Copy,"Copywriter, advertising",Corporate investment banker,Counselling psychologist,Counsellor,Curator,Cytogeneticist,Dance movement psychotherapist,Dancer,Data scientist,Database administrator,Dealer,"Designer, ceramics/pottery","Designer, exhibition/display","Designer, furniture","Designer, industrial/product","Designer, interior/spatial","Designer, jewellery","Designer, multimedia","Designer, television/film set","Designer, textile","Development worker, community","Development worker, international aid",Diagnostic radiographer,Dispensing optician,"Doctor, general practice","Doctor, hospital",Drilling engineer,Early years teacher,Ecologist,Economist,"Editor, commissioning","Editor, film/video","Editor, magazine features",Education administrator,"Education officer, community","Education officer, museum",Educational psychologist,Electrical engineer,Electronics engineer,"Embryologist, clinical",Emergency planning/management officer,Energy engineer,Energy manager,"Engineer, aeronautical","Engineer, agricultural","Engineer, automotive","Engineer, biomedical

In [51]:
# 4. Dividing the dataset

X = df.drop('is_fraud', axis=1)
y = df['is_fraud']

In [52]:
X.shape, y.shape

((1296675, 613), (1296675,))

In [53]:
X.head()

,amt,gender,city_pop,entertainment,food_dining,gas_transport,grocery_net,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,Academic librarian,"Accountant, chartered","Accountant, chartered certified","Accountant, chartered public finance",Accounting technician,Acupuncturist,Administrator,"Administrator, arts","Administrator, charities/voluntary organisations","Administrator, education","Administrator, local government",Advertising account executive,Advertising account planner,Advertising copywriter,Advice worker,Aeronautical engineer,Agricultural consultant,Aid worker,Air broker,Air cabin crew,Air traffic controller,Airline pilot,Ambulance person,Amenity horticulturist,Analytical chemist,Animal nutritionist,Animal technologist,Animator,Applications developer,Arboriculturist,Archaeologist,Architect,Architectural technologist,Archivist,Armed forces logistics/support/administrative officer,Armed forces technical officer,Armed forces training and education officer,Art gallery manager,Art therapist,Artist,Arts development officer,Associate Professor,Audiological scientist,Barista,Barrister,Barrister's clerk,"Biochemist, clinical",Biomedical engineer,Biomedical scientist,Bookseller,Broadcast engineer,Broadcast journalist,Broadcast presenter,Building control surveyor,Building services engineer,Building surveyor,"Buyer, industrial","Buyer, retail",Cabin crew,Call centre manager,Camera operator,Careers adviser,Careers information officer,Cartographer,Catering manager,Ceramics designer,Charity fundraiser,Charity officer,Chartered accountant,Chartered legal executive (England and Wales),Chartered loss adjuster,Chartered public finance accountant,Chemical engineer,"Chemist, analytical",Chief Executive Officer,Chief Financial Officer,Chief Marketing Officer,Chief Operating Officer,Chief Strategy Officer,Chief Technology Officer,Chief of Staff,Child psychotherapist,Chiropodist,Civil Service administrator,Civil Service fast streamer,"Civil engineer, contracting",Claims inspector/assessor,Clinical biochemist,Clinical cytogeneticist,Clinical psychologist,Clinical research associate,Clothing/textile technologist,Colour technologist,Commercial horticulturist,Commercial/residential surveyor,Commissioning editor,Communications engineer,Community arts worker,Community development worker,Community education officer,Community pharmacist,Company secretary,Comptroller,"Conservation officer, historic buildings","Conservator, furniture","Conservator, museum/gallery",Contracting civil engineer,Contractor,Control and instrumentation engineer,Copy,"Copywriter, advertising",Corporate investment banker,Counselling psychologist,Counsellor,Curator,Cytogeneticist,Dance movement psychotherapist,Dancer,Data scientist,Database administrator,Dealer,"Designer, ceramics/pottery","Designer, exhibition/display","Designer, furniture","Designer, industrial/product","Designer, interior/spatial","Designer, jewellery","Designer, multimedia","Designer, television/film set","Designer, textile","Development worker, community","Development worker, international aid",Diagnostic radiographer,Dispensing optician,"Doctor, general practice","Doctor, hospital",Drilling engineer,Early years teacher,Ecologist,Economist,"Editor, commissioning","Editor, film/video","Editor, magazine features",Education administrator,"Education officer, community","Education officer, museum",Educational psychologist,Electrical engineer,Electronics engineer,"Embryologist, clinical",Emergency planning/management officer,Energy engineer,Energy manager,"Engineer, aeronautical","Engineer, agricultural","Engineer, automotive","Engineer, biomedical","Engine

In [54]:
y.value_counts()

is_fraud
0    1289169
1       7506
Name: count, dtype: int64

In [64]:
chunk_size = 100  # Number of rows per chunk
X_converted_chunks = []

for start in range(0, X.shape[0], chunk_size):
    end = start + chunk_size
    chunk = X[start:end].astype('float32')
    X_converted_chunks.append(chunk)

# Combine the processed chunks
X_converted = pd.concat(X_converted_chunks, axis=0)


In [65]:
# Handling imbalanced dataset
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy=0.1)
X, y = smote.fit_resample(X_converted, y)

In [67]:
y.value_counts()

is_fraud
0    1289169
1     128916
Name: count, dtype: int64

In [ ]:
# 6. Data normalization
X.describe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X[:] = scaler.fit_transform(X)

In [ ]:
X.head()

In [ ]:
# 7. Train - Test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
# 8. Building the ANN model

import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    #input  layer + hidden layer 1
    keras.layers.Dense(300, input_shape=(542,), activation='relu'),
    #hidden layer 2
    keras.layers.Dense(150, activation='relu'),
    # output layer
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=200)

In [ ]:
# 9. Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy:', accuracy)


In [ ]:
model.summary()

In [ ]:
# To make predictions
y_pred = model.predict(X_test)
y_pred[:5]

In [ ]:
binary_predictions = (y_pred > 0.5).astype(int)

In [ ]:
binary_predictions[:5]

In [ ]:
y_test[:5]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, binary_predictions))
print(classification_report(y_test, binary_predictions))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cf = confusion_matrix(y_test, binary_predictions)
sns.heatmap(cf, annot=True, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')

In [ ]:
# 10. Saving the model
model.save('../models/fraud_model.h5')
